In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
# Add parent dir to path, so that python finds the lenskit package
sys.path.insert(0,parentdir)

In [2]:
from lenskit.metrics import dataGenerator
from lenskit import batch, topn, util, topnFair
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, funksvd, user_knn as uknn, item_knn as iknn 
from lenskit import topn, topnFair

In [3]:
import numpy as np
import pandas as pd
import math
import pickle
%matplotlib inline

In [4]:
dataset = os.path.join(parentdir,'book_data/ratings.csv')     
ratings = pd.read_csv(dataset, sep=',',
                      names=['user', 'item', 'rating'], header=0)

data = os.path.join(parentdir,'book_data/book_author_gender.csv')

items = pd.read_csv(data, sep=',',
                      names=['item','gender'], header=0)

In [5]:
ratings.head()


,user,item,rating
0,156050,102337377,8.0
1,102032,107971879,5.0
2,14422,104568501,9.0
3,122037,101204681,8.0
4,109864,919256170,7.0


In [6]:
items = items.loc[(items["gender"] == "male") | (items["gender"] == "female")]
items = items.reset_index(drop="true")
print(items.shape)
items.head()

(4251558, 2)


,item,gender
0,113484902,male
1,100940351,male
2,104633344,female
3,202193849,male
4,101354697,male


In [7]:
ratings_joined =  pd.merge(ratings, items.set_index('item'),how="inner", on='item')

In [12]:
ratings_joined.shape

(319988, 4)

In [8]:
#rateditems = ratings.item.unique()
#rateditems = pd.DataFrame(rateditems, columns =['item'])
#rateditems = pd.merge(items.set_index('item'), on='item', how="inner"  )
#rateditems = pd.merge(rateditems, items.set_index('item'),how="inner", on='item')
#print(rateditems.shape)
#rateditems = rateditems.dropna()
#print(rateditems.shape)
#rateditems.head()




(96680, 2)
(96680, 2)


,item,gender
0,107971879,male
1,101204681,female
2,102130480,male
3,103178071,female
4,101910165,female


In [15]:
ratings_joined.groupby('user').item.count()
#ratings.groupby('user').item.count()

user
8          5
9          1
14         2
16         1
17         3
19         1
22         1
26         2
32         1
39         2
42         1
44         1
51         1
53         4
56         2
64         1
67         1
69         1
70         1
73         1
75         1
78         1
81         1
82         1
83         1
85         1
86         1
87         2
88         1
91         1
          ..
278724     1
278732     1
278740     1
278750     1
278755     1
278759     1
278760     1
278772     1
278773     4
278777     1
278778     1
278782     2
278786     1
278798     2
278800     1
278807     1
278818     2
278820     1
278824     1
278825     1
278828     1
278831     1
278832     3
278836     1
278843    16
278844     2
278846     1
278851    10
278852     1
278854     6
Name: item, Length: 63724, dtype: int64

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96680 entries, 0 to 96679
Data columns (total 2 columns):
item      96680 non-null int64
gender    96680 non-null object
dtypes: int64(1), object(1)
memory usage: 2.2+ MB
